## Full Length Only Mega-genome
```
Lead     : ababaian / RCE
Issue    : n/a
start    : 2020 05 17
complete : 2020 05 --
files    : s3://serratus-public/seq/flom1/
notebook : s3://serrauts-public/notebook/200517_flom1/
```

## Introduction
> rce:
>
> I took the opportunity to remove many short mega sequences which might make focus on Cov and automated detection more difficult, e.g. segmented genomes where each segment is quite short.
> Flom1 contains 673 full-length Covs and 184 full-length genomes from other families which have "complete genome" in their defline.
> The shortest sequences is now NC_007558 Adult diarrheal rotavirus strain J19 at 649nt.

See: [FLOM Note](https://github.com/ababaian/serratus/wiki/FLOM-reference)


As a secondary objective this serves a pilot test for scheduler stress testing with Jeff's recent updates.

### Files from RCE
- `cb6b66b38a1494baba77b9c640053f4e  flom1.tz`
- `295f6bb10fbffc3fce344c9d2b3d9b8a  flom1_sumzer_metadata.tz`

Moved to S3: `s3://serratus-public/seq/flom1/`


## Materials and Methods


In [ ]:
# EC2 Instance Commands:
# Build/Run `serratus-align`container for indexing
sudo yum install -y docker
sudo yum install -y git
sudo service docker start

#export DOCKERHUB_USER='serratusbio'
#sudo docker login

git clone https://github.com/ababaian/serratus.git; cd serratus/containers
./build_containers

sudo docker run --rm --entrypoint /bin/bash -it serratus-align:latest

In [ ]:
# local bedtools install
wget https://github.com/arq5x/bedtools2/releases/download/v2.29.2/bedtools.static.binary
mv bedtools.static.binary bedtools
chmod 755 bedtools; mv bedtools /usr/bin/

In [ ]:
# Local usearch install
#The clustered database was made with usearch:
wget https://drive5.com/downloads/usearch11.0.667_i86linux32.gz
gzip -dc usearch11.0.667_i86linux32.gz > usearch
chmod 755 usearch; mv usearch /usr/bin/usearch


In [1]:
# Local Dustmasker install
cd /home/serratus/
wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ncbi-blast-2.10.0+-x64-linux.tar.gz
tar -xvf ncbi-blast-2.10.0+-x64-linux.tar.gz
cp ncbi-blast-2.10.0+/bin/* /usr/bin/


[sudo] password for artem: 


## Initial version (deprecated)

In [ ]:
# bash inside `serratus-align`
mkdir flom1; cd flom1

# Start from flom1 - post 99% identity filtering
aws s3 cp --recursive s3://serratus-public/seq/flom1/ ./

# untar
tar -xvf flom1.tz 
tar -xvf flom1_sumzer_metadata.tz 

```
bash-4.2# md5sum *
921b0a8f024f80c5df7decf5e455cc02  acc_len_taxid.txt
3bcb7034e6a5492393161d8bb485109b  flom1.fa
cb6b66b38a1494baba77b9c640053f4e  flom1.tz
f9017589cf471c47d8c541e886065b22  flom1_acc_desc.tsv
d97857f8fcdf8d8cf08864fab6b46dd2  flom1_acc_desc_cov.tsv
4860ef9c5ce481e4a58cdbc38a8020ad  flom1_acc_desc_not_cov.tsv
edfaeffc8182bf3e56dd293a1f88dc59  flom1_acc_length.tsv
96990c8cdd3a07071224364c7a4f728b  flom1_accs.tsv
295f6bb10fbffc3fce344c9d2b3d9b8a  flom1_sumzer_metadata.tz
1b43f071d99de081af25bb39394509f5  taxid_desc.txt
bce281c0b7bb5563dfd1167c1a943e85  taxids.txt

```

In [ ]:
# Remove Accessions on blacklist
# Strips headers to accessions only
# wc -l cov0.id99.fa
# 514753 cov0.id99.fa

# Does nothing in this case
#seqkit grep flom1.fa -i -r -v \
#  -p KC786228 -p AX191447 -p AX191449 \
#  -p FB764528 -p HV449436 -p CS382036 \
#  -p FV537213 -p AY204705 -p AF209745 \
#  -p AY204704 \
#  > flom1.bl.fa


In [ ]:
INPUTFA='flom1.fa'

# Create polyNT masks (10-X seed)
seqkit locate --bed -i -m 0 -p 'AAAAAAAAAA' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyAT.mask.bed

seqkit locate --bed -i -m 0 -p 'GGGGGGGGGG' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyGC.mask.bed

cat polyAT.mask.bed polyGC.mask.bed > \
  polyNT.bed

rm polyAT.mask.bed polyGC.mask.bed poly10.bed poly10.sort.bed

In [ ]:
## Manually set blacklisted regions
#echo -e "JB181528.1\t3111\t3307" >> blacklist.bed
#echo -e "CS460762.1\t37177\t37211" >> blacklist.bed
#echo -e "CS460762.1\t30166\t30243" >> blacklist.bed
#echo -e "CS480537.1\t37170\t37220" >> blacklist.bed
#echo -e "CS480537.1\t30166\t30241" >> blacklist.bed
#echo -e "MK562374.1\t474\t542" >> blacklist.bed
#echo -e "DL231478.1\t43\t2296" >> blacklist.bed
#
## new for cov2m
#echo -e "KC786228.1\t49\t191 " >> blacklist.bed

In [ ]:
INPUTFA='flom1.fa'

# Soft mask low complexity regions via dustmasker
dustmasker -in $INPUTFA \
  -window 30 -outfmt interval \
  -out flom1.dust
# output ... 
# >KF600647.1
# 2840 - 2846
# 30101 - 30114
# >MK129253.1
# 2845 - 2851
# 30106 - 30149
# >CS480537.1
# 22517 - 22524
# 22569 - 22575
# 29227 - 29246
# ...

# Convert interval dust file to bed file
while read -r line; do
  if [ $(echo $line | head -c 1) = ">" ]; then
    header=$( echo $line | sed 's/>//g' - )
  else
    start=$(echo $line | cut -f1 -d' ' -)
    end=$(echo $line | cut -f3 -d' ' - )
    echo -e "$header\t$start\t$end" >> dust.bed
  fi
done < flom1.dust

In [ ]:
# Blacklist, polyNT and dust mask

cat polyNT.bed blacklist.bed dust.bed > mask.regions.bed

rm polyNT.bed blacklist.bed dust.bed

In [ ]:
INPUTFA='flom1.fa'

# cov2 pan-genome
# Soft-masked pan-genome
bedtools maskfasta -fi $INPUTFA \
  -bed mask.regions.bed -fo flom1.softmasked.fa -soft
 
# Hard-masked pan-genome
bedtools maskfasta -fi $INPUTFA \
  -bed mask.regions.bed -fo flom1.hardmasked.fa -mc N

In [ ]:
mv flom1.fa flom1.unmasked.fa
mv flom1.hardmasked.fa flom1.fa

# Count each fasta file
wc -l *.fa 

```
bash-4.2# wc -l *.fa 
  290275 flom1.fa
  290275 flom1.softmasked.fa
  290275 flom1.unmasked.fa
  870825 total

```

In [ ]:
# Remove intermediates
rm *.tsv taxids.txt flom1.dust

# Compress stuff we don't need immediatly
gzip flom1.softmasked.fa flom1.unmasked.fa

In [ ]:
# Build bowtie2 + faidx index for cov1r.fa
bowtie2-build --threads 4 --seed 1337 flom1.fa flom1
samtools faidx flom1.fa

```
921b0a8f024f80c5df7decf5e455cc02  acc_len_taxid.txt
4f23f4250a8ce8ad3fb27afff742a769  flom1.1.bt2
d1b52624722a101ba7cd36296300d0ff  flom1.2.bt2
f1fb4f5124f738c2e29af5aef55957a9  flom1.3.bt2
c0ee65df374a298672671447792e1555  flom1.4.bt2
dabd68e0533e57be018947d25e911e35  flom1.fa
c835e906e20de75290765d8a863b0125  flom1.fa.fai
bc2fe2003ed752796ce18cf113a8507d  flom1.rev.1.bt2
60f6bb7a8f114bbf24b0b841f280e37f  flom1.rev.2.bt2
06c0bdc89cd06142b8195071b448fa72  flom1.softmasked.fa.gz
cb6b66b38a1494baba77b9c640053f4e  flom1.tz
a62ff6db8159a6af9e73f30260c36207  flom1.unmasked.fa.gz
295f6bb10fbffc3fce344c9d2b3d9b8a  flom1_sumzer_metadata.tz
270e4e6100f91e3fa7e8b833bf436650  mask.regions.bed
1b43f071d99de081af25bb39394509f5  taxid_desc.txt
```

In [ ]:
# Upload to s3 public access area
aws s3 sync ./ s3://serratus-public/seq/flom1/

# Hotfix - Forgot to dustmask regions

doh!

In [ ]:
# bash inside `serratus-align`
mkdir flom1; cd flom1

# Start from flom1
aws s3 cp s3://serratus-public/seq/flom1/flom1.tz  ./
aws s3 cp s3://serratus-public/seq/flom1/flom1_sumzer_metadata.tz   ./

# untar
tar -xvf flom1.tz 
tar -xvf flom1_sumzer_metadata.tz 

In [ ]:
# Remove Accessions on blacklist
# Strips headers to accessions only
# wc -l cov0.id99.fa
# 514753 cov0.id99.fa

# Does nothing in this case
#seqkit grep flom1.fa -i -r -v \
#  -p KC786228 -p AX191447 -p AX191449 \
#  -p FB764528 -p HV449436 -p CS382036 \
#  -p FV537213 -p AY204705 -p AF209745 \
#  -p AY204704 \
#  > flom1.bl.fa


In [ ]:
## Manually set blacklisted regions

# Camelpox Homology
echo -e "NC_003391\t9477\t9698" >> blacklist.bed
# BVDV Transmembrane homology
echo -e "NC_001461\t4990\t5268" >> blacklist.bed
# Mollusk Virus homology to GPX1 gene
echo -e "NC_001731\t83019\t83523" >> blacklist.bed

In [ ]:
INPUTFA='flom1.fa'

# Short Window Dust Masking ---------------------
# Soft mask low complexity regions via dustmasker
dustmasker -in $INPUTFA \
  -window 30 -outfmt interval \
  -out flom1.dust30

# Convert interval dust file to bed file
while read -r line; do
  if [ $(echo $line | head -c 1) = ">" ]; then
    header=$( echo $line | sed 's/>//g' - )
  else
    start=$(echo $line | cut -f1 -d' ' -)
    end=$(echo $line | cut -f3 -d' ' - )
    echo -e "$header\t$start\t$end" >> dust30.bed
  fi
done < flom1.dust30


# Long Window Dust Masking ---------------------
# Soft mask low complexity regions via dustmasker
dustmasker -in $INPUTFA \
  -window 64 -outfmt interval \
  -out flom1.dust64

# Convert interval dust file to bed file
while read -r line; do
  if [ $(echo $line | head -c 1) = ">" ]; then
    header=$( echo $line | sed 's/>//g' - )
  else
    start=$(echo $line | cut -f1 -d' ' -)
    end=$(echo $line | cut -f3 -d' ' - )
    echo -e "$header\t$start\t$end" >> dust64.bed
  fi
done < flom1.dust64

echo ''

In [ ]:
INPUTFA='flom1.fa'

# Create polyNT masks (10-X seed)
# FLOM seems to be missing any NT tract >7nt
seqkit locate --bed -i -m 0 -p 'AAAAAAAAAA' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyAT.mask.bed

seqkit locate --bed -i -m 0 -p 'GGGGGGGGGG' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyGC.mask.bed

cat polyAT.mask.bed polyGC.mask.bed > \
  polyNT.bed

rm polyAT.mask.bed polyGC.mask.bed poly10.bed poly10.sort.bed

In [ ]:
# Combine blacklist, nt mask and dustmask
cat polyNT.bed blacklist.bed dust30.bed dust64.bed > mask.tmp.bed
  sort -k1,1 -k2,2n mask.tmp.bed > mask.sort.bed
  bedtools merge -i mask.sort.bed > mask.regions.bed
rm mask.sort.bed mask.tmp.bed

wc -l *.bed

rm polyNT.bed blacklist.bed dust30.bed dust64.bed

```                     
     3 blacklist.bed
   650 dust30.bed
  1974 dust64.bed
  2187 mask.regions.bed
     0 polyNT.bed
  4814 total
```

In [ ]:
mv flom1.fa flom1.unmasked.fa
mv flom1.hardmasked.fa flom1.fa

# Count each fasta file
wc -l *.fa 

```
  290275 flom1.fa
  290275 flom1.softmasked.fa
  290275 flom1.unmasked.fa
  870825 total
```

In [ ]:
# Remove intermediates
rm *.tsv taxids.txt flom1.dust*

# Compress stuff we don't need immediatly
gzip flom1.softmasked.fa flom1.unmasked.fa

In [ ]:
# Build bowtie2 + faidx index for cov1r.fa
bowtie2-build --threads 4 --seed 1337 flom1.fa flom1
samtools faidx flom1.fa

md5sum *

```
921b0a8f024f80c5df7decf5e455cc02  acc_len_taxid.txt
90ea2df6019a8dbf3b021772d2a3791b  flom1.1.bt2
a0fed29b358eda3a5f3cfc57b71d3952  flom1.2.bt2
038e2f95345291ca4925299c71be4cae  flom1.3.bt2
7416b1f3a2c801b593183fc9a47c7bf8  flom1.4.bt2
43146c18064e1fecf5f39ba0ec6b1ac7  flom1.fa
c835e906e20de75290765d8a863b0125  flom1.fa.fai
c048a407634a501fdafdd91658964b80  flom1.rev.1.bt2
b813ef4577475ee03c8a737d30b4edbf  flom1.rev.2.bt2
a9f623b3eb2bb9754bc6c4ec2e42f824  flom1.softmasked.fa.gz
cb6b66b38a1494baba77b9c640053f4e  flom1.tz
a62ff6db8159a6af9e73f30260c36207  flom1.unmasked.fa.gz
295f6bb10fbffc3fce344c9d2b3d9b8a  flom1_sumzer_metadata.tz
130d19850877275d55ea730343126a34  mask.regions.bed
1b43f071d99de081af25bb39394509f5  taxid_desc.txt
```

In [ ]:
# Updated summarizer meta-data
aws s3 cp s3://serratus-public/seq/flom1/flom1.sumzer.tsv ./
md5sum flom1.sumzer.tsv

```
b424362d91020bab1f7b63b6da976aeb  flom1.sumzer.tsv
```

In [ ]:
aws s3 sync ./ s3://serratus-public/seq/flom1/